# Modelflow brings together a model and a Pandas DataFrame
**ModelFlow** is designed to combine a **model specification** with a **Pandas DataFrame** that contains the corresponding data.  
In this setup, each **column** of the DataFrame represents a model variable, while the **row index** represents time.  
This structure allows ModelFlow to evaluate and simulate the model directly on the data, period by period.  

While Pandas provides powerful tools for managing time series, it does not natively handle **lagged variables** or **simultaneous equation systems** in a convenient way.  
ModelFlow fills this gap by extending the DataFrame framework with capabilities for **lag and lead referencing**, **recursive and simultaneous solution algorithms**, and **automatic dependency handling** between variables.  

In this way, ModelFlow bridges the gap between **data storage** and **model computation**, making it possible to define, solve, and analyze complex dynamic systems directly within the familiar Python and Pandas environment.


In [74]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [75]:
from modelmanipulation import Mexplode

## Specification of a model 
In its simplest form, the ModelFlow domain-specific language (DSL) consists of **formulas**, each defining a relationship between variables.  
However, ModelFlow also supports a **higher-level DSL** that extends this basic syntax with Python-based text processing features.  
This enables the user to **generate formulas programmatically**, allowing the model specification itself to be written at a **higher level of abstraction**.  

With this extended DSL, users can define **templates or patterns** for equations that apply across multiple entities — for example, all sectors in a macroeconomic model or all banks in a stress-testing framework — and then automatically expand them into a complete set of detailed, entity-specific equations.  

This approach dramatically reduces **boilerplate text** and improves **consistency** across the model.  
Instead of manually writing large blocks of repetitive equations, the modeler can rely on Python constructs such as loops, lists, and string formatting to generate the full specification dynamically and transparently.  

By integrating this higher-level DSL directly into the modeling framework, ModelFlow combines the **clarity and expressiveness** of a concise formula language with the **flexibility and automation** capabilities of Python — making it possible to specify large, structured models in a compact, readable, and reproducible way.


## Example Models  
To illustrate the use of ModelFlow and its formula-based DSL, we present two simple examples.  
The first is a small macroeconomic model describing the relationship between **GDP** and **consumption**.  
The second is a **debt dynamics** model that tracks the evolution of government debt over time.  
Together, they demonstrate how behavioral and accounting identities can be expressed concisely in ModelFlow’s syntax.  



### 1. GDP and Consumption Model  

This simple model links output $Y_t$ and consumption $C_t$ over time.  
Consumption depends on lagged disposable income, and output is determined by aggregate demand.  


\begin{align*}
C_t &= \alpha_0 + \alpha_1 Y_{t} + \alpha_2 Y_{t-1} + \varepsilon_t^C \\
Y_t &= C_t + I_t + G_t + X_t - M_t
\end{align*}


- $C_t$ – consumption at time \$t\$  
- $Y_t$ – output (GDP) at time \$t\$  
- $I_t, G_t, X_t, M_t$ – investment, government spending, exports, and imports  
- $\alpha_0, \alpha_1\ , \alpha_2$ – behavioral parameters  
- $\varepsilon_t^C$ – consumption shock  

This captures **lagged adjustment behavior** (consumption depends on past income) while treating the other demand components as **exogenous**. Also the model is contains **contemporaneous feedback loops** so it has to be solved as a system.   

To input this into ModelFlow the two formulas can be written as: 


```
Frml <stoc> C = a0 +  a1*Y + a2*Y(-1) + epsC $
Frml Y = C + I + G + X - M $
```

---

### 2. Debt Dynamics Model  

A second example tracks the evolution of government debt over time.  
Debt increases with the primary deficit and with interest payments on existing debt.  


\begin{aligned}
D_t &= (1 + i_{t-1}) D_{t-1} + PD_t \\
PD_t &= G_t - T_t
\end{aligned}

- \$D_t\$ – government debt at time \$t\$  
- \$i_{t-1}\$ – interest rate on debt from the previous period  
- \$PD_t\$ – primary deficit (spending minus taxes)  
- \$G_t\$ – government spending  
- \$T_t\$ – taxes  

Here, \$D_t\$ is **endogenous**, while \$i_t\$, \$G_t\$, and \$T_t\$ may be treated as **exogenous inputs** or determined by other model blocks.  

This simple dynamic ensures that current debt depends on **lagged debt** and **current fiscal flows**, a common structure in fiscal and macroeconomic models.  

To input this into ModelFlow the two formulas can be written as: 

```
Frml D  = (1 + i(-1))*D(-1) + PD $
Frml PD = G - T $
```


\begin{align*}
y_t^1  &=  f^1(y_{t+u}^1...,y_{t+u}^n...,y_t^2...,y_{t}^n...y_{t-r}^1...,y_{t-r}^n,x_t^1...x_{t}^k,...x_{t-s}^1...,x_{t-s}^k) \\
y_t^2  &=  f^2(y_{t+u}^1...,y_{t+u}^n...,y_t^1...,y_{t}^n...y_{t-r}^1...,y_{t-r}^n,x_t^1...x_{t}^k,...x_{t-s}^1...,x_{t-s}^k) \\
\vdots \\
y_t^n  &=  f^n(y_{t+u}^1...,y_{t+u}^n...,y_t^1...,y_{t}^{n-1}...y_{t-r}^1...,y_{t-r}^n,x_t^1...x_{t}^r,x..._{t-s}^1...,x_{t-s}^k)
\end{align*}

where $ y_t^1 $ is one of n endogenous variables and $x_t^1$ is one of k exogenous variables. To have a solution the system must have as many equations as there are unknown (endogenous variables).

Substituting the variable mnemonics Y,C,I,G,X,M for the y's and x's above, a simple macrostructural model can be written as as a system of 6 equations in 6 unknowns:

\begin{align*}
Y_t  &=  C_t+I_t+G+t+ (X_t-M_t) \\
C_t &= c(C_{t-1},C_{t-2},I_t,G_t,X_t,M_t,P_t)\\
I_t &= i(I_{t-1},I_{t-2},C_t,G_t,X_t,M_t,P_t)\\
G_t &= g(G_{t-1},G_{t-2},C_t,I_t,X_t,M_t,P_t)\\
X_t &= x(X_{t-1},X_{t-2},C_t,I_t,G_t,M_t,P_t,P^f_t)\\
M_t &= m(M_{t-1},M_{t-2},C_t,I_t,G_t,X_t,P_t,P^f_t)
\end{align*}
 
Where $Y_t$ is an identity and $C_t, I_t, G_t, X_t, M_t$ are behavioral variables and $P_t, P^f_t$ (domestic and foreign prices, respectively) are exogenous in this simple model. 


## A Model  

For the purpose of this note, a (simple) model like the two mentioned above can be thought of as a system of equations that links **endogenous** and **exogenous** variables over time.  

- **Endogenous variables** are those determined *within* the model — their values are the outcome of the relationships the model describes.  
- **Exogenous variables**, on the other hand, are determined *outside* the model — they are treated as given inputs or shocks that influence the endogenous variables but are not themselves affected by them.  

In time-based models, variables can appear with **lags** or **leads**.  
A *lag* refers to the value of a variable in a previous period — for example, \$y_{t-1}\$ is the value of \$y\$ one period before \$t\$.  
A *lead* refers to a future value — for example, \$y_{t+1}\$ represents the value of \$y\$ one period after \$t\$.  

Including lags allows the model to capture persistence and delayed effects in economic relationships, such as habit formation or gradual adjustment.  
In this note, only **lags** will be used, as they are sufficient for representing the backward-looking relationships typical of most empirical and accounting-based models.  
However, **ModelFlow** also allows the use of *leaded* variables, which are useful in models with rational or model-consistent expectations.  


Lets say we have:
 - **n**: number of endogenous variables  
 - **k**: number of exogenous variables  
 - **r**: maximum lag of endogenous variables  
 - **s**: maximum lag of exogenous variables  
 - **t**: time index (year, quarter, month, day, or any other unit)  


\begin{aligned}
y_t^1  &= \;f^1(\; y_t^2,\ldots,y_t^n \;\;,\;\;y_{t-1}^1,\ldots,y_{t-r}^n\;\;,\;\;x_t^1,\ldots,x_t^k\;\;,\;\;x_{t-1}^1,\ldots,x_{t-s}^k \;) \\
y_t^2  &= \;f^2(\; y_t^1,\ldots,y_t^n \;\;,\;\;y_{t-1}^1,\ldots,y_{t-r}^n\;\;,\;\;  x_t^1,\ldots,x_t^k\;\;,\;\; x_{t-1}^1,\ldots,x_{t-s}^k \;) \\
       & \;\;\vdots \\
y_t^n  &= f^n\bigl(\; 
   \underbrace{\underbrace{y_t^1,\ldots,y_t^{n-1}}_{\text{current}} \;,\;
               \underbrace{y_{t-1}^1,\ldots,y_{t-r}^n}_{\text{lagged}}}_{\text{endogenous}},\;
   \underbrace{\underbrace{x_t^1,\ldots,x_t^k}_{\text{current}} \;,\;
               \underbrace{x_{t-1}^1,\ldots,x_{t-s}^k}_{\text{lagged}}}_{\text{exogenous}}
   \;\bigr)
\end{aligned}


Sometime such models are implemented directly in a programming environment such as Python, Matlab, or even Excel, where they are both *specified* and *solved* in the same code.  

---

## Why Separate Specification and Implementation?  

Instead of mixing economic reasoning with programming details, a model can be written in a **domain-specific language (DSL)** designed specifically for economics and finance. Separating specification from implementation has several advantages:  

1. **Clarity and focus** – The specification remains close to the underlying economic logic, expressed in a concise and readable form. The modeler concentrates on *what the relationships are*, not on *how the computer solves them*.  

2. **Expressiveness** – The DSL uses syntax familiar to economists (equations, lags, variables), avoiding the clutter of general-purpose programming constructs.  

3. **Maintainability** – Models can be extended or updated without rewriting solution code. Changing an equation is as simple as editing one line in the specification.  

4. **Portability** – Once specified, the same model can be executed with different numerical engines or platforms without altering its economic content.  

5. **Transparency** – The separation makes models easier to review, validate, and communicate, since the specification reads like economic theory rather than computer code.  

This approach is common in several econometric systems such as **EViews**, **Dynare**, and **GEKKO**, which all use their own DSLs to describe models.  

**ModelFlow** follows a similar philosophy but with a key difference: it does not introduce a new language for data handling, visualization, or model analysis. Instead, ModelFlow relies on the **Python ecosystem** - primarily `pandas`, but also `networkx`, `sympy`, `numpy`, and other libraries - allowing the model specification to be combined seamlessly with powerful tools for data manipulation, result analysis, and visualization.  


## A Simple Specification Language  

In practice, the DSL for economic models is built around equation-like statements. For example:  
```
Frml <stoc> C = a0 + a1*Y(-1) + epsC $
Frml Y = C + I + G + X - M $

```

Here each formula (`Frml`) defines the relationship between variables in a way that mirrors how economists usually write them, with lags (`(-1)`) and simple algebraic notation.  

The optional `<tag>` is used for transforming the frml or to control solving - more on that later.

This kind of business-logic-oriented specification acts as the “blueprint” of the model, leaving the task of solving it to the underlying implementation engine.  


## Expanding the Language  

The basic `Frml` statements form the **lowest level** of the ModelFlow domain-specific language (DSL), defining the direct relationships between variables.  
To make the language more expressive, concise, and suitable for large, structured models, ModelFlow extends this foundation with several higher-level constructs:  

- **Lists and sublists** – to organize entities such as banks, sectors, or countries, and automatically expand equations across these groups.  
- **Do ... enddo loops** – to generate sets of equations programmatically over the members of a list or over a specified range.  
- **Sum operator** – to express aggregation directly within equations by summing over lists or sublists.  
- **Doable equations** – shorthand notation combining loops and summation, simplifying the definition of repeated or aggregated relationships.  
- **Stripped-down syntax** – allows equations to be written without explicit `Frml` and `$` markers, resulting in cleaner, more compact specifications.  

Together, these features make it possible to write models at a **higher level of abstraction**, reducing repetition and improving consistency across model components.  
Instead of defining each equation manually, the modeler can use lists, loops, and operators to express economic logic once and have ModelFlow automatically generate all the necessary detailed equations.  
This design allows the DSL to scale from small illustrative examples to large, multi-entity macroeconomic and financial models while maintaining clarity and transparency.  


### Mexplode – Expanding the Stripped-Down DSL  

The **ModelFlow expanded DSL** allows modelers to specify relationships at a high level, using constructs such as **lists**, **doable equations**, and the **stripped-down syntax** (where formulas are written without explicit `Frml` or `$` markers).  
To make these concise specifications executable, ModelFlow uses a preprocessor called **`Mexplode`**.

`Mexplode` reads the stripped-down model specification and **automatically expands** it into the full set of `Frml` statements that ModelFlow can interpret and solve.  

It recognizes high-level patterns such as `do` blocks, `sum` operators, and **list dimensions** embedded in variable names (identified by double underscores `__`).  
For example, a variable like `profit__bank__sector` tells `Mexplode` that the model should be expanded across the lists `BANKS` and `SECTORS`.  

This automated expansion eliminates the need for the modeler to write repetitive, low-level code.  
A single compact template equation can be transformed into hundreds of consistent, entity-specific formulas.  

In essence, `Mexplode` acts as the **bridge between model specification and execution**:  
it takes a concise, human-readable input written in the extended DSL and generates the complete, detailed set of equations needed for computation.

`Mexplode` exposes a set of **computed properties** that make it easy to inspect or use the processed model directly:  

- **`show`** — Displays the resulting expanded and normalized `Frml` statements for inspection.  
- **`mmodel`** — Returns a fully constructed **ModelFlow model instance** ready for simulation, analyses and visualisation. But remember simulation requires a relevant dataframe. 


**Key benefits of Mexplode:**
- Processes **stripped-down DSL** directly — no need to write explicit `Frml` or `$`.  
- Automatically detects and expands **list dimensions** from variable naming conventions (`__`).  
- Handles **doable equations**, **sum operators**, and **conditional expansions** transparently.  
- Produces a complete, executable model that maintains full consistency with the high-level specification.

By automating this expansion step, `Mexplode` allows modelers to focus entirely on the **economic logic** of their models rather than the mechanics of their implementation.


### Mexplode Properties  

**For use by the very interested modeller**, the `Mexplode` dataclass stores the full transformation state of a model as it moves from the stripped-down DSL to the fully expanded, normalized set of `Frml` statements ready for ModelFlow.  
Each field corresponds to a specific stage in this expansion and normalization process.  

| **Property** | **Type** | **Description** |
|:---------------|-----------|:-----------------|
| `original_statements` | `str` | Input model specification written in the stripped-down DSL. |
| `normal_frml` | `str` | Output containing the normalized model equations. |
| `normal_main` | `str` | Normalized `Frml` statements for the main model equations. |
| `normal_fit` | `str` | Normalized `Frml` statements for fitted-value calculations. |
| `normal_calc_add` | `str` | Normalized `Frml` statements for add-factor calculations. |
| `clean_frml_statements` | `str` | Intermediate version with properly formatted `Frml` lines and structured list syntax. |
| `post_doable` | `str` | Model after expansion of all `doable` equations. |
| `post_do` | `str` | Model after expansion of explicit `do ... enddo` loops. |
| `post_sum` | `str` | Model after expansion of all summation (`sum`) operators. |
| `expanded_frml` | `str` | Fully expanded set of executable `Frml` equations. |
| `normal_expressions` | `List[Any]` | List of parsed normalized expression objects. |
| `list_specification` | `str` | String representation of all list definitions found in the model. |
| `modellist` | `str` | Dictionary-like string of defined lists and their attributes. |
| `funks` | `List[Any]` | List of user-defined functions that can be referenced in model formulas. |

These internal fields allow the modeller to follow, in detail, how the stripped-down DSL is gradually transformed into executable ModelFlow syntax.  
They provide a complete diagnostic view of the intermediate stages—useful for debugging, educational purposes, or extending ModelFlow’s preprocessing pipeline.  

The higher-level properties such as **`show`** and **`mmodel`**—described in the main `Mexplode` section—build on this internal structure to present the final expanded model or to produce a fully functional ModelFlow model instance.


### Lists  

Among the higher-level constructs, **lists** and **sublists** are central to making ModelFlow scalable and expressive.  
They allow the modeler to define **groups of entities** (such as banks, portfolios, sectors, or countries) and to attach **attributes** or **hierarchies** to those entities.  
This enables equations to be written once as templates and then **expanded automatically** across all relevant list members.  

Sublists can be used to define logical relationships between entities — for example, linking each bank to its home country, or associating portfolios with regulatory categories.  
These structures make it possible to organize models systematically and to maintain consistency across entities that share the same behavioral equations.  

This approach eliminates redundancy and manual repetition: instead of writing dozens or hundreds of similar equations, the modeler defines a **single equation pattern**, and ModelFlow expands it dynamically over the members of a list.  

This capability is especially valuable in:  

- **Stress-testing models** – where many banks, each with multiple portfolios, must be represented consistently.  
- **Macroeconomic models** – where sectors of the economy (households, firms, government, rest of the world, etc.) share common behavioral structures.  
- **Cross-country or panel models** – where the same specification is applied across multiple countries or regions.  

#### Syntax  

List definitions in the ModelFlow DSL follow a simple and consistent format:  

- Each list **starts** with the keyword `list`.  
- It **ends** with a `$`.  
- It can contain one or more **sublists**, separated by `/`.  
- Each sublist **starts** with a `name:` followed by its members.  


Below are a few examples of list definitions:  

In [76]:
lists ='''
list BANKS = BANKS    : Danske  Nordea Ibs /
             bankcountry : denmark sweden denmark  

list country = country : denmark sweden 

list ports = ports  : households   NFC    RE  sov /
               reg  : AIRB  AIRB   STA  STA 


'''

So the list statement starts with a list and ends with a $ sublists are seperated by / and each starts with a `sublist name:` 

Meaning


- `list BANKS = ...` defines a top-level list of banks with an **sublist** `bankcountry` that specifies where each bank belongs.  
- `list country = ...` creates a country list.  
- `list ports = ...` groups portfolios (`households`, `NFC`, `RE`, `sov`) and creates **sublists** with regulatory categories (`AIRB`, `STA`).  
- `list country__danske`, `list country__nordea`, and `list country__ibs` are **lists**, linking each bank to its loan countries.  

The benefits of lists: 

- **Compactness** – groups can be declared once and reused throughout the model.  
- **Flexibility** – adding or removing members requires only updating the list, not the equations.  
- **Transparency** – clear structure makes it easy to see which entities and attributes the model covers.  

By defining lists in this way, ModelFlow provides a structured layer between the economic logic and the generated equations, enabling users to manage complex multi-entity models efficiently and transparently.

### Do ... Enddo Loops  

The **`do ... enddo`** construct allows repetitive patterns of equations to be generated automatically.  
When used together with **lists**, it enables the modeler to create a complete set of equations for every member of a list without having to write each one manually.  

A `do` block defines a **loop** over the members of a specified list (for example, all banks, sectors, or countries).  
Within the block, the loop variable can be referenced directly inside formulas — ModelFlow will automatically substitute the appropriate member name during expansion.  

This mechanism is essential for large-scale models where the same behavioral relationships apply to multiple entities.  
It ensures **consistency**, reduces **repetition**, and makes model specifications more **maintainable** and **transparent**.  

#### Syntax  

The syntax of a `do` block is straightforward:  

- The loop **starts** with `do <list>`  
- It **ends** with `enddo`  
- One or more `Frml` statements are written inside the block  
- ModelFlow automatically expands each `Frml` for every member of the list  

#### Example  

In [77]:
Mexplode(lists +
''' 
do banks 
     profit__{banks} = revenue__{banks} - expenses__{banks} 
 enddo  
''').show

FRML <> PROFIT__DANSKE = REVENUE__DANSKE-EXPENSES__DANSKE $
FRML <> PROFIT__NORDEA = REVENUE__NORDEA-EXPENSES__NORDEA $
FRML <> PROFIT__IBS = REVENUE__IBS-EXPENSES__IBS $


### Conditional Loops
The do statement can also include conditions that restrict the loop to members of a list matching specific attribute values.
This is written using the syntax do <list> <sublist>=<value>.

For example, suppose the list BANKS includes a sublist bankcountry,
then the following block will only expand for banks located in Denmark:

In [78]:
Mexplode(lists +
''' 
do banks bankcountry = denmark
     profit__{banks} = revenue__{banks} - expenses__{banks} 
 enddo  
''').show

FRML <> PROFIT__DANSKE = REVENUE__DANSKE-EXPENSES__DANSKE $
FRML <> PROFIT__IBS = REVENUE__IBS-EXPENSES__IBS $


ModelFlow will expand this block only for banks whose bankcountry attribute equals denmark.

This conditional looping makes it possible to write targeted relationships for specific subsets of entities 
for instance, defining different equations for domestic and foreign banks, or applying alternative behavioral rules to different sectors or regions.

By combining lists, sublists, and conditional do loops, ModelFlow allows highly structured models to be specified compactly while preserving clarity and flexibility.

### Doable Equations  

While `do ... enddo` loops provide an explicit way to expand formulas across members of a list,  
the **`doable`** construct offers a more concise and automatic alternative.  

A **doable equation** is a *high-level shorthand* that combines the functionality of loops and summations without requiring the user to write repetitive loop structures.  
Instead of manually specifying `do` blocks, ModelFlow can **detect the relevant lists automatically** by inspecting the **first** variable names in the equation.  

Variables that contain double underscores (`__`) indicate **list dimensions**.  
For example, in `profit__{banks}`, ModelFlow recognizes that `profit` is a variable defined over the list `BANKS`.  
Similarly, in `loss__{banks}__{ports}`, the two underscores identify two list dimensions — `BANKS` and `PORTS`.  

#### Doable summary

$$
\texttt{doable} \;
\underbrace{\langle tag \rangle}_{\text{Tag}} \;
\underbrace{[list\;\; sublist=value,...]}_{\text{Optional Conditions}} \;
\underbrace{%
   \underbrace{\texttt{VARIABLE}}_{\text{LHS Expression}}
   \underbrace{\_\_\{list1\}\_\_\{list2\}}_{\text{LHS Dimensions}}
}_{\text{LHS}} \;
= \;
\underbrace{\texttt{Expression including sublists}}_{\text{RHS Expression}}
$$


When ModelFlow encounters a `doable` equation such as:  

In [79]:
Mexplode(lists +'doable profit__{banks} = revenue_{banks} - expenses_{banks}').show   


FRML <> PROFIT__DANSKE = REVENUE_DANSKE-EXPENSES_DANSKE $
FRML <> PROFIT__NORDEA = REVENUE_NORDEA-EXPENSES_NORDEA $
FRML <> PROFIT__IBS = REVENUE_IBS-EXPENSES_IBS $


it automatically expands the formula across all members of the list BANKS — just as a do BANKS ... enddo loop would.

Thus, the doable keyword eliminates the need for explicit looping syntax, making the specification both shorter and easier to read.

Doable equations can also include conditions and tags in the same way as do loops.
For example:

In [80]:
Mexplode(lists+'doable [banks bankcountry=denmark] profit__{banks}=revenue__{banks}-expenses__{banks}').show 


FRML <> PROFIT__DANSKE = REVENUE__DANSKE-EXPENSES__DANSKE $
FRML <> PROFIT__IBS = REVENUE__IBS-EXPENSES__IBS $


expands only for banks where the sublist bankcountry equals denmark.

This mechanism is particularly powerful when dealing with complex, multi-dimensional systems — such as stress-test models with many banks and portfolios —
since the modeler can write compact, pattern-based equations that ModelFlow expands automatically by detecting all relevant list dimensions.

In short, doable equations serve as an elegant, high-level syntax that:

 - Avoids the boilerplate of do ... enddo structures,

 - Automatically determines which lists apply by analyzi ng variable dimensions, and

 - Keeps model specifications concise, readable, and scalable.

### Summing of variables 

In [81]:
Mexplode(lists+'loss_danske =  sum(ports,loss__danske__{ports}))').show 

FRML <> LOSS_DANSKE = (LOSS__DANSKE__HOUSEHOLDS+LOSS__DANSKE__NFC+LOSS__DANSKE__RE+LOSS__DANSKE__SOV)) $


In [82]:
Mexplode(lists+'loss_total =  sum(banks,sum(ports,loss__{banks}__{ports}))').show 

FRML <> LOSS_TOTAL = ((LOSS__DANSKE__HOUSEHOLDS+LOSS__DANSKE__NFC+LOSS__DANSKE__RE+LOSS__DANSKE__SOV)+(LOSS__NORDEA__HOUSEHOLDS+LOSS__NORDEA__NFC+LOSS__NORDEA__RE+LOSS__NORDEA__SOV)+(LOSS__IBS__HOUSEHOLDS+LOSS__IBS__NFC+LOSS__IBS__RE+LOSS__IBS__SOV)) $


In [83]:
Mexplode(lists+ 'loss_total_sta =  sum(banks,sum(ports reg=STA,loss__{banks}__{ports}))').show

FRML <> LOSS_TOTAL_STA = ((LOSS__DANSKE__RE+LOSS__DANSKE__SOV)+(LOSS__NORDEA__RE+LOSS__NORDEA__SOV)+(LOSS__IBS__RE+LOSS__IBS__SOV)) $


In [84]:
Mexplode(lists+
'doable <sum=total>  profit__{banks}=revenue__{banks}-expenses__{banks}'
        ).show


FRML <SUM=TOTAL> PROFIT__DANSKE = REVENUE__DANSKE-EXPENSES__DANSKE $
FRML <SUM=TOTAL> PROFIT__NORDEA = REVENUE__NORDEA-EXPENSES__NORDEA $
FRML <SUM=TOTAL> PROFIT__IBS = REVENUE__IBS-EXPENSES__IBS $
FRML <SUM=TOTAL> PROFIT__TOTAL = (PROFIT__DANSKE+PROFIT__NORDEA+PROFIT__IBS) $


In [85]:
Mexplode(lists+
'doable <sum=total> [banks bankcountry=denmark] profit__{banks}=revenue__{banks}-expenses__{banks}'
        ).show


FRML <SUM=TOTAL> PROFIT__DANSKE = REVENUE__DANSKE-EXPENSES__DANSKE $
FRML <SUM=TOTAL> PROFIT__IBS = REVENUE__IBS-EXPENSES__IBS $
FRML <SUM=TOTAL> PROFIT__TOTAL = (PROFIT__DANSKE+PROFIT__IBS) $
